In [1]:
#usual imports
import pandas as pd
import numpy as np
import os
import sys
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib 
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
print(os.listdir('../Social'))

['filtered_Monthly_data.csv', 'Untitled.ipynb', 'CMO_MSP_Mandi.csv', 'Price Comparison.ipynb', 'Filter Outliers.ipynb', 'Seasonality_type.csv', 'filtered_cmo_mandi.csv', 'Deseasonalized_price.csv', 'Detect Seasonality type and deseasonalize prices.ipynb', '.ipynb_checkpoints', 'Monthly_data_cmo.csv']


In [2]:
#imoporting dataset
monthly = pd.read_csv('filtered_Monthly_data.csv')
print(monthly.head(10))

   Unnamed: 0        APMC      Commodity  Year  Month  arrivals_in_qtl  \
0           0  Ahmednagar          Bajri  2015  April               79   
1           1  Ahmednagar          Bajri  2016  April              106   
2           2  Ahmednagar  Wheat(Husked)  2015  April             1253   
3           3  Ahmednagar  Wheat(Husked)  2016  April              387   
4           4  Ahmednagar  Sorgum(Jawar)  2015  April             3825   
5           5  Ahmednagar  Sorgum(Jawar)  2016  April             2093   
6           6  Ahmednagar          Maize  2015  April               75   
7           7  Ahmednagar          Maize  2016  April              155   
8           8  Ahmednagar           Gram  2015  April             1794   
9           9  Ahmednagar           Gram  2016  April              630   

   min_price  max_price  modal_price     date district_name   state_name  
0       1406       1538         1463  2015-04    Ahmadnagar  Maharashtra  
1       1788       1925         187

### A Little Cleaning

In [3]:
monthly['date'] = pd.to_datetime(monthly['date']) # conversion to date time format
# making apmc-commodity clusters
monthly['cluster'] = monthly['APMC'] + '-' + monthly['Commodity']
#dropping modal prices having zero values as they can be a problem while creating the multiplicative model
monthly = monthly[monthly['modal_price'] != 0].reset_index(drop = True)
#dropping the extra index column which was added previously 
monthly = monthly.drop(['Unnamed: 0'], axis = 1)
print(monthly.columns)

Index(['APMC', 'Commodity', 'Year', 'Month', 'arrivals_in_qtl', 'min_price',
       'max_price', 'modal_price', 'date', 'district_name', 'state_name',
       'cluster'],
      dtype='object')


### Selecting those (APMC - Commodity) Clusters where we have data for at least a whole year and filtering the dataset accordingly

In [4]:
monthly_ = monthly[['APMC', 'Commodity', 'modal_price', 'Year', 'cluster']].groupby(['APMC', 'Commodity', 'Year', 'cluster'], as_index = False).count().rename(columns = {'modal_price':'count'}).reset_index(drop = True)


In [5]:
monthly_


,APMC,Commodity,Year,cluster,count
0,Aamgaon,PADDY-UNHUSKED,2016,Aamgaon-PADDY-UNHUSKED,1
1,Aamgaon,Paddy-Unhusked,2014,Aamgaon-Paddy-Unhusked,4
2,Aamgaon,Paddy-Unhusked,2015,Aamgaon-Paddy-Unhusked,12
3,Aamgaon,Paddy-Unhusked,2016,Aamgaon-Paddy-Unhusked,10
4,Aarni,Black Gram,2015,Aarni-Black Gram,1
5,Aarni,Black Gram,2016,Aarni-Black Gram,2
6,Aarni,Gram,2014,Aarni-Gram,2
7,Aarni,Gram,2015,Aarni-Gram,6
8,Aarni,Gram,2016,Aarni-Gram,5
9,Aarni,Green Gram,2015,Aarni-Green Gram,2


In [6]:
#dropping the columns where we dont have the data for a whole year
monthly_ = monthly_[monthly_['count'] == 12].reset_index(drop = True)

In [7]:
monthly_

,APMC,Commodity,Year,cluster,count
0,Aamgaon,Paddy-Unhusked,2015,Aamgaon-Paddy-Unhusked,12
1,Akhadabalapur,Soybean,2015,Akhadabalapur-Soybean,12
2,Akluj,Bitter Gourd,2015,Akluj-Bitter Gourd,12
3,Akluj,Brinjal,2015,Akluj-Brinjal,12
4,Akluj,Cabbage,2015,Akluj-Cabbage,12
5,Akluj,Chikoo,2015,Akluj-Chikoo,12
6,Akluj,Chillies(Red),2015,Akluj-Chillies(Red),12
7,Akluj,Ladies Finger,2015,Akluj-Ladies Finger,12
8,Akluj,Maize,2015,Akluj-Maize,12
9,Akluj,Onion,2015,Akluj-Onion,12


In [8]:
# Now cleaning the data to have only those cleaned values as described previously
cluster_unique  = monthly_['cluster'].unique().tolist()
monthly = monthly[monthly['cluster'].isin(cluster_unique)].reset_index(drop = True)

In [9]:
monthly.to_csv('cleaned_monthly_data.csv', index = None)
print(monthly)

                 APMC         Commodity  Year      Month  arrivals_in_qtl  \
0               Akole             Bajri  2015      April               52   
1               Akole             Bajri  2016      April               75   
2               Akole    Paddy-Unhusked  2015      April              113   
3               Akole    Paddy-Unhusked  2016      April              448   
4               Akole     Wheat(Husked)  2015      April              164   
5               Akole     Wheat(Husked)  2016      April              115   
6               Akole              Gram  2015      April               39   
7               Akole              Gram  2016      April               70   
8               Akole           Soybean  2015      April              285   
9               Akole           Soybean  2016      April              378   
10            Jamkhed             Bajri  2015      April              164   
11            Jamkhed             Bajri  2016      April               45   

### Now creating a function which first selects a particular (apmc-commodity) cluster then it decomposes the data according to both models (i.e a multiplicative and an additive model) then checks for the autocrrelation function of the residual part of the decomposition. The seasonality type corresponds to the model which gives the higher acf for the residual part. It also deseasonalizes the modal price according to the type of the seasonality a cluster follows

In [17]:
def check_seasonality(df, deseasonalize = False):
    df['period'] = df['Year'].astype('str') + df['Month']
    df['period'] = pd.to_datetime(df['period'], format = '%Y%B')
    
    #creating a temporary dataframe with modal price as values and months - year as indices
    tsdf = pd.DataFrame(data = df['modal_price'].values, index = df['period'], columns = ['modal_price'])
    tsdf.index = pd.to_datetime(tsdf.index)
    
    #resampling the data according to months
    tsdf = tsdf.resample('MS').mean()
    tsdf = tsdf.fillna(int(tsdf['modal_price'].mean())) #just a precaution so we dont get any missing values whatsoever
    
    #decomposition of the series according to both models
    result_add = seasonal_decompose(tsdf, model = 'additive', freq = 12)
    result_mul = seasonal_decompose(tsdf, model = 'multiplicative', freq = 12)
    
    # calculating autocorrelation function and accounting for less data in the except block
    try:
        additive_acf = sum(np.asarray(acf(result_add.resid, missing = 'drop'))*2)
        mult_acf = sum(np.asarray(acf(result_mul.resid, missing = 'drop'))*2)
        
    except:
        result_add = seasonal_decompose(tsdf, model='additive',freq=6)
        result_mul = seasonal_decompose(tsdf, model='multiplicative',freq=6)
        
        additive_acf=sum(np.asarray(acf(result_add.resid, missing='drop'))*2)
        mult_acf=sum(np.asarray(acf(result_mul.resid, missing='drop'))*2)
        
        
    if additive_acf < mult_acf:
        value = 'Additive'
        if deseasonalize:
            tsdf['Seasonal'] = result_add.seasonal.values # extracting the seasonal values
            values = tsdf['modal_price'] - tsdf['Seasonal'] # deseasonalization of price for additive model
            values = values.tolist()
            
            indices = df.index.tolist()
            values_range = range(0, len(values))
            for index, value in zip(indices, values_range):
                df.at[index, 'deseasonalize_price'] = values[value]
            return df[['APMC', 'Commodity', 'cluster', 'date', 'modal_price',  'deseasonalize_price']]
        else:
            return 'Additive'
        
        
    else:
        value = 'Multiplicative'
        if deseasonalize:
            tsdf['Seasonal'] = result_mul.seasonal.values
            values = tsdf['modal_price']/tsdf['Seasonal'] # deseasonalization of price for Multiplicative model
            values = values.tolist()
            
            indices = df.index.tolist()
            values_range = range(0, len(values))
            for index, value in zip(indices, values_range):
                df.at[index, 'deseasonalize_price'] = values[value]
            return df[['APMC', 'Commodity', 'cluster', 'date', 'modal_price', 'deseasonalize_price']]
        else:
            return 'Multiplicative'
            
    
                        

### now using the above made function to get seasonality type and getting the deseasonalized price

In [11]:
monthly['date'] = pd.to_datetime(monthly['date'])
i = 0 
seasonality = []
for cluster in cluster_unique:
    df = monthly[monthly['cluster'] == cluster]
    seasonality.append((cluster, check_seasonality(df)))
    i += 1
    if i % 200 == 0:
        print('completed upto index', i)

completed upto index 200
completed upto index 400
completed upto index 600
completed upto index 800


In [12]:
seasonality

[('Aamgaon-Paddy-Unhusked', 'Multiplicative'),
 ('Akhadabalapur-Soybean', 'Additive'),
 ('Akluj-Bitter Gourd', 'Multiplicative'),
 ('Akluj-Brinjal', 'Multiplicative'),
 ('Akluj-Cabbage', 'Multiplicative'),
 ('Akluj-Chikoo', 'Additive'),
 ('Akluj-Chillies(Red)', 'Multiplicative'),
 ('Akluj-Ladies Finger', 'Additive'),
 ('Akluj-Maize', 'Additive'),
 ('Akluj-Onion', 'Additive'),
 ('Akluj-Potato', 'Additive'),
 ('Akluj-Shevga', 'Additive'),
 ('Akluj-Tomato', 'Multiplicative'),
 ('Akluj-Water Melon', 'Multiplicative'),
 ('Akole-Bajri', 'Multiplicative'),
 ('Akole-Gram', 'Multiplicative'),
 ('Akole-Paddy-Unhusked', 'Multiplicative'),
 ('Akole-Soybean', 'Multiplicative'),
 ('Akole-Wheat(Husked)', 'Additive'),
 ('Alibag-Rice(Paddy-Hus)', 'Multiplicative'),
 ('Amarawati-Black Gram', 'Additive'),
 ('Amarawati-Green Gram', 'Additive'),
 ('Amarawati-Sorgum(Jawar)', 'Additive'),
 ('Amarawati-Soybean', 'Multiplicative'),
 ('Amarawati-Wheat(Husked)', 'Additive'),
 ('Ambejogai-Gram', 'Additive'),
 ('A

In [13]:
seasonality = pd.DataFrame(seasonality, columns = ['Cluster', 'Seasonality'])
seasonality.to_csv('Seasonality_type.csv', index = False)

In [18]:
s_dic = seasonality.set_index('Cluster').to_dict()['Seasonality']
monthly['deseasonalized_price'] = np.nan
df = pd.DataFrame()
for cluster in s_dic.keys():
    temp = check_seasonality(monthly[monthly['cluster'] == cluster], deseasonalize = True)
    df = pd.concat([df, temp])

In [19]:
df

,APMC,Commodity,cluster,date,modal_price,deseasonalize_price
6624,Aamgaon,Paddy-Unhusked,Aamgaon-Paddy-Unhusked,2015-04-01,1412,1539.551920
6625,Aamgaon,Paddy-Unhusked,Aamgaon-Paddy-Unhusked,2016-04-01,1657,1693.188631
6628,Aamgaon,Paddy-Unhusked,Aamgaon-Paddy-Unhusked,2015-08-01,1562,1524.016891
6629,Aamgaon,Paddy-Unhusked,Aamgaon-Paddy-Unhusked,2016-08-01,1667,1501.760841
6632,Aamgaon,Paddy-Unhusked,Aamgaon-Paddy-Unhusked,2014-12-01,1518,1576.033672
6633,Aamgaon,Paddy-Unhusked,Aamgaon-Paddy-Unhusked,2015-12-01,1564,1523.967698
6636,Aamgaon,Paddy-Unhusked,Aamgaon-Paddy-Unhusked,2015-02-01,1505,1431.565103
6637,Aamgaon,Paddy-Unhusked,Aamgaon-Paddy-Unhusked,2016-02-01,1558,1411.914442
6640,Aamgaon,Paddy-Unhusked,Aamgaon-Paddy-Unhusked,2015-01-01,1533,1476.385097
6641,Aamgaon,Paddy-Unhusked,Aamgaon-Paddy-Unhusked,2016-01-01,1523,1478.489326


In [20]:
df.to_csv('Deseasonalized_price.csv', index = False)